In [128]:
import numpy as np
import datetime
import csv
import matplotlib.pyplot as plt
import urllib
import json


np.random.seed(1)

# columns in order
# Body Water % (FROM THE NEXT DAY), WATER DRANK (GRAMS), ACTIVITY TIME, AVERAGE TEMP, AVERAGE HUMIDITY, SCORE
data = np.zeros((365, 5)) - 1
month_to_str = {"January": 1, "February": 2, "March":3, "April":4, "May":5, "June": 6, "July":7, \
                "August":8, "September":9, "October": 10, "November": 11, "December": 12}



previous = datetime.datetime(2021, 1, 1)
index  = 0


with open("Andriola daily servings summary 210101 thru 211231.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    next(reader)
    for row in reader:
        if not row[8]:
            continue
            
        month, day, year = [int(element) for element in row[0].split("/")]
        current_date = datetime.datetime(year, month, day)
        if current_date == previous:
            data[index, 1] += float(row[8]) 
            
        else:
            difference = current_date - previous
            index += difference.days
            previous = current_date
            data[index, 1] = float(row[8])

previous = datetime.datetime(2021, 1, 1)
index  = 0

with open("arboleaf-Tom Andriola 2020-01 to 2022-06.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    next(reader)
    for row in reader:
        if not row[8]:
            continue
        
        month, day, year, *other = row[0].split(" ")
        if year != "2021":
            continue
            
        current_date = datetime.datetime(int(year), month_to_str[month], int(day[:-3]))
        difference = current_date - previous
        index += difference.days
        previous = current_date
        data[index, 0] = float(row[8])
        
previous = datetime.datetime(2021, 1, 1)
index  = 0
with open("oura_2020-04-14_2022-06-11_trends.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    next(reader)
    for row in reader:
        year, month, day  = [int(element) for element in row[0].split("-")]
        if year != 2021:
            continue
            
        current_date = datetime.datetime(year, month, day)
        difference = current_date - previous
        index += difference.days
        previous = current_date
        data[index, 2] = sum(int(element) for element in row[39:42])
        
        
# interpolate most of the missing bodywater percentages to increase

locations = np.where(data[:,0] != -1)[0]

for i in range(len(locations) - 1):
    findex = locations[i]
    sindex = locations[i + 1]
    
    start, end = data[findex, 0], data[findex, 0]
    rise = end - start
    run = sindex - findex

    for i, index in enumerate(range(findex, sindex)):
        data[index, 0] = ((rise / run) * i) + start
        
        
# DATA HAS BEEN BUILT< NOW NEED TO PULL WEATHER API DATA
# GENERATE A LIST OF DAYS I NEED

days = []

start = datetime.datetime(2021, 1, 1)

for i, row in enumerate(data):
    if row[0] > 0 and row[1] > 0 and row[2] > 0:
        new_day = start + datetime.timedelta(days=i)
        days.append(new_day)
        
# select 250 days so i need less api calls to generate the rest of the data
days = np.random.choice(days, 250, False)
days= np.sort(days)
# Get weather data corresponding to these points

base_url = "http://api.weatherstack.com/historical"
access_key = "1de3076806ab8c2a6fb4457865d61627"
query = "Irvine"
to_write = open("DATA.csv", "w")

for day in days:
    i = day - start
    date = day.strftime("%Y-%m-%d")
    params = {
        "access_key": access_key,
        "query" : query,
        "historical_date": date,
        "hourly": "1"
    }
    params = urllib.parse.urlencode(params)
    req = urllib.request.Request(base_url + "?" + params)
    with urllib.request.urlopen(req) as f:
        weather_json = json.load(f)
        weather_json = weather_json["historical"][date]
        
        data[i.days, 3] =  weather_json["avgtemp"]
        
        humidities = [element["humidity"] for element in weather_json["hourly"]]            
        data[i.days, 4] =  sum(humidities) / len(humidities)
        
    to_write.write(date+"," + ",".join([str(element) for element in data[i.days]]) + "\n")
        
to_write.close()

        
